In [1]:
import pandas as pd
import numpy as np
import torch

In [4]:
ten = torch.exp(torch.tensor(0.03))
print(ten)
ten.item()
print(type(ten.item()))

tensor(1.0305)
<class 'float'>


In [4]:
sst2 = pd.read_csv("sst2_dataset.txt", sep="\t")

In [5]:
sst2.head(3)

,sentence_index,sentence
0,1,The Rock is destined to be the 21st Century 's...
1,2,The gorgeously elaborate continuation of `` Th...
2,3,Effective but too-tepid biopic


In [6]:
len(sst2)

11855

In [9]:
sst2.iloc[2]["sentence"]

'Effective but too-tepid biopic'

In [10]:
wikitext = pd.read_csv("wiki.valid.raw", sep="\t")

In [11]:
wikitext.head(3)

,= Homarus gammarus =
0,"Homarus gammarus , known as the European lobs..."
1,= = Description = =
2,"Homarus gammarus is a large crustacean , with..."


In [12]:
len(wikitext)

2460

In [13]:
sst2 = sst2["sentence"]
sst2.head(3)

0    The Rock is destined to be the 21st Century 's...
1    The gorgeously elaborate continuation of `` Th...
2                       Effective but too-tepid biopic
Name: sentence, dtype: object

In [14]:
sst2.to_csv("sst2_preprocessed.txt", index=False, header=False)

In [16]:
sst2.iloc[0]

"The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal ."

In [18]:
msk = np.random.rand(len(sst2)) < 0.8
train = sst2[msk]
test = sst2[~msk]
print(len(train))
train.head(2)

9472


0    The Rock is destined to be the 21st Century 's...
1    The gorgeously elaborate continuation of `` Th...
Name: sentence, dtype: object

In [19]:
print(len(test))

2383


In [20]:
train.to_csv("sst2_train.txt", index=False, header=False)
test.to_csv("sst2_test.txt", index=False, header=False)

In [ ]:
plm = GPT2LMHeadModel.from_pretrained("models/" + model_name, pad_token_id = tokenizer.eos_token_id)

In [36]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, DistilBertForMaskedLM


# model = AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased")
model = DistilBertForMaskedLM.from_pretrained(
    "gpt2_fine_tuned_sst2",
    local_files_only=True
)

In [37]:
model

DistilBertForMaskedLM(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): L

In [38]:
device = torch.device("cuda")
model = model.to(device)

In [30]:
model.is_cuda

AttributeError: 'collections.OrderedDict' object has no attribute 'is_cuda'

In [2]:
torch.cuda.is_available()

False

In [1]:
from transformers import DistilBertTokenizer

In [3]:
import torch
torch.cudaDeviceReset()

AttributeError: module 'torch' has no attribute 'cudaDeviceReset'